In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
%%time
# I like to disable my Notebook Warnings.
import warnings
warnings.filterwarnings('ignore')


In [ ]:
%%time
# Notebook Configuration...

# Amount of data we want to load into the Model...
DATA_ROWS = None
# Dataframe, the amount of rows and cols to visualize...
NROWS = 50
NCOLS = 15
# Main data location path...
BASE_PATH = '...'


In [ ]:
%%time
# Configure notebook display settings to only use 2 decimal places, tables look nicer.
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', NCOLS) 
pd.set_option('display.max_rows', NROWS)


In [ ]:
%%time
# Load the CSVs into a pandas dataframe for future data manipulation.
trn_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')
tst_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')

sub = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')


In [ ]:
%%time
trn_data.info()


In [ ]:
%%time
trn_data.head()


In [ ]:
%%time
trn_data.describe()


In [ ]:
%%time
trn_data.isnull().sum().sum()


In [ ]:
%%time
trn_data.isnull().sum()


In [ ]:
%%time
trn_data.nunique()


In [ ]:
trn_data.nunique().sort_values(ascending = True)


In [ ]:
%%time
categ_cols = ['f_29','f_30','f_13', 'f_18','f_17','f_14','f_11','f_10','f_09','f_15','f_07','f_12','f_16','f_08','f_27']
trn_data[categ_cols].sample(5)


In [ ]:
%%time
correlation = trn_data.corr()


In [ ]:
%%time
correlation


In [ ]:
%%time
correlation['target'].sort_values(ascending = False)[:5]


In [ ]:
%%time
correlation['target'].sort_values(ascending = True)[:5]


In [ ]:
%%time
trn_data['target'].value_counts()


In [ ]:
%%time
trn_data['target'].describe()


In [ ]:
%%time
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
def encode_features(df, cols = ['f_27']):
    for col in cols:
        df[col + '_enc'] = encoder.fit_transform(df[col])
    return df

trn_data = encode_features(trn_data)
tst_data = encode_features(tst_data)


In [ ]:
trn_data.head()


In [ ]:
%%time
ignore = ['id', 'target', 'f_27'] # f_27 has been label encoded...

features = [feat for feat in trn_data.columns if feat not in ignore]
target_feature = 'target'


In [ ]:
%%time
from sklearn.model_selection import train_test_split
test_size_pct = 0.10
X_train, X_valid, y_train, y_valid = train_test_split(trn_data[features], trn_data[target_feature], test_size = test_size_pct, random_state = 42)


In [ ]:
%%time
from xgboost  import XGBClassifier


In [ ]:
%%time
params = {'n_estimators': 4096,
          'max_depth': 8,
          'learning_rate': 0.15,
          'subsample': 0.95,
          'colsample_bytree': 0.95,
          'reg_lambda': 1.50,
          'reg_alpha': 1.50,
          'gamma': 1.50,
          'random_state': 46,
          'objective': 'binary:logistic',
          'tree_method': 'gpu_hist',
         }


In [ ]:
%%time
xgb = XGBClassifier(**params)
xgb.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], eval_metric = ['auc'], early_stopping_rounds = 128, verbose = 250)


In [ ]:
%%time
def plot_feature_importance(importance, names, model_type, max_features = 10):
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_df = fi_df.head(max_features)

    #Define size of bar plot
    plt.figure(figsize=(8,6))
    
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')


In [ ]:
%%time
import seaborn as sns
import matplotlib.pyplot as plt
plot_feature_importance(xgb.feature_importances_,X_train.columns,'XG BOOST ', max_features = 15)


In [ ]:
%%time
sub.head()


In [ ]:
%%time
from sklearn.metrics import roc_auc_score
preds = xgb.predict_proba(tst_data[features])[:, 1]


In [ ]:
%%time
sub['target'] = preds
sub.to_csv('my_submission_043022.csv', index = False)


In [ ]:
%%time
sub.head()
